# Solving Sudokus using mathematical programming
> "Sudoku is a logic-based, combinatorial number-placement puzzle. The objective is to fill a 9×9 grid with digits so that each column, each row, and each of the nine 3×3 subgrids that compose the grid contains all of the digits from 1 to 9. The puzzle setter provides a partially completed grid, which for a well-posed puzzle has a single solution." [Wikipedia](https://en.wikipedia.org/wiki/Sudoku)

So how can we solve this with math programming? Well, ultimately we have to make discrete *decisions* about what numbers to put which fields. So let's get started:
## Initialization and data import

In [1]:
import xpress as xp
import numpy as np
from dataclasses import dataclass
from sys import stdout

# The data
sudoku = ( (0, 0, 0,  0, 9, 0,  1, 0, 0),
           (2, 8, 0,  0, 0, 5,  0, 0, 0),
           (7, 0, 0,  0, 0, 6,  4, 0, 0),

           (8, 0, 5,  0, 0, 3,  0, 0, 6),
           (0, 0, 1,  0, 0, 4,  0, 0, 0),
           (0, 7, 0,  2, 0, 0,  0, 0, 0),

           (3, 0, 0,  0, 0, 1,  0, 8, 0),
           (0, 0, 0,  0, 0, 0,  0, 5, 0),
           (0, 9, 0,  0, 0, 0,  0, 7, 0) )

@dataclass(frozen=True)
class Value:
    row: int
    column: int
    value: int
        
model = xp.problem("Sudoku")
n = 9 # The grid size
nRange = range(9)
s = int(np.sqrt(n)) # The sub-cell size
sRange = range(s)

# Create the list of values
values = [Value(row, column, value) for row in nRange for column in nRange for value in nRange]

## Defining the variables
The only thing we need here is the binary variables we discussed above:

In [140]:
x = {v : xp.var(vartype = xp.binary, name = f'x_{v.row}, {v.column}, {v.value}') for v in values}
model.addVariable(x)

## The constraints
To solve a Sudoku, we need to "explain" the rules to the model. 

1. Each cell must take exactly one value: ($\sum \limits_{v} x_{i,j,v} = 1$)

In [141]:
each_cell_one_value = (xp.constraint(xp.Sum(x[v] for v in values 
                                            if v.row == row and 
                                            v.column == column) == 1, 
                                     name=f'Different value in {row},{column}') 
                       for row in nRange for column in nRange)

2. Each value is used exactly once per row: $\sum \limits_{i} x_{i,j,v} = 1$

In [142]:
each_value_once_per_row = (xp.constraint(xp.Sum(x[v] for v in values 
                                                if v.value == value and 
                                                v.column == column) == 1, 
                                     name=f'{value} once per row for column {column}') 
                           for value in nRange for column in nRange)

3. Each value is used exactly once per column: $\sum \limits_{j} x_{i,j,v} = 1$

In [143]:
each_value_once_per_column = (xp.constraint(xp.Sum(x[v] for v in values 
                                                   if v.row == row and 
                                                   v.value == value) == 1, 
                                     name=f'{value} once per column for row {row}') 
                              for row in nRange for value in nRange)

4. Each value is used exactly once per 3x3 subgrid:
\begin{equation}
\sum \limits_{i\in r*[1,3]} \sum \limits_{j\in c*[1,3]} x_{i,j,v} = 1, \hspace{0.15cm} r \in [1,3], c\in [1,3]
\end{equation}

In [144]:
def get_val(values: list, i: int, j: int, v: int):
    return [val for val in values if val.row == i and val.column == j and val.value == v].pop()

each_value_per_subgrid = (xp.constraint(xp.Sum(x[get_val(values, i,j,val)] for i in range(r*s,(r+1)*s) for j in range(c*s,(c+1)*s)) == 1, 
                                        name=f'{val} for subgrid {r},{c}') for r in sRange for c in sRange for val in nRange)

## Give the 'start solution'
If we were just to solve this, we would get any solution: we need to fix the initial solution to get the unique solution.

In [145]:
starting_solution = (xp.constraint(x[v] == 1, 
                                   name =f'Fix {v.value} for {v.row} and {v.column}')
                     for v in values if sudoku[v.row][v.column] == v.value + 1)

model.addConstraint(each_cell_one_value, each_value_once_per_row, each_value_once_per_column, each_value_per_subgrid, starting_solution)

## Solve the problem and print solution

In [146]:
model.solve()

In [147]:
xVal = {v : model.getSolution(x[v]) for v in values}

""" Print Sudoku grid """
for i in nRange:
    if (i > 0):
        stdout.write('\n')
    if (i % 3 == 0):
        stdout.write('\n')
    for j in nRange:
        stdout.write('   ' if (j % 3 == 0) else ' ')
        for v in nRange:
            if (xVal[get_val(values,i,j,v)] == 1):
                stdout.write(str(v+1))
stdout.write('\n')


   4 5 6   3 9 7   1 2 8
   2 8 3   1 4 5   9 6 7
   7 1 9   8 2 6   4 3 5

   8 2 5   9 1 3   7 4 6
   6 3 1   5 7 4   8 9 2
   9 7 4   2 6 8   5 1 3

   3 4 7   6 5 1   2 8 9
   1 6 2   7 8 9   3 5 4
   5 9 8   4 3 2   6 7 1
